<a href="https://colab.research.google.com/github/jamesETsmith/2022_simons_collab_pyscf_workshop/blob/main/demos/04_Energy_vs_Lattice_Constant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up the Jupyter notebook

* We need to install a few things before we get started
  * [PySCF](https://pyscf.org/) (for the quantum chemsitry)
  * [py3DMol](https://3dmol.csb.pitt.edu/) for visualizing the molecule
  * [plotly](https://plotly.com/python/) and kaleido for plotting

In [ ]:
%pip install -q pyscf py3DMol plotly kaleido